In [1]:
# -*- coding: utf-8 -*- 
import pandas as pd

from crawling import crawl as cr
import utilfunc as ut
import noun_proc as pr


#######################
### PLEASE TYPE URL ###  
#######################

default_url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=009&aid=0004704439'
# url = input("TYPE THE URL OF NAVER NEWS: ")

def main(URL, ST1, ST2, ST3, ST4, k):
    boo, title, article, comments, likes, dislikes, replys = cr.crawling(URL)
    if boo:
        dfIn = sub(title, article, comments, likes, dislikes, replys, k)
        dfIn[6] = ST1*dfIn[2] + ST2*dfIn[3] + ST3*dfIn[4] + ST4*dfIn[5]
        result = dfIn[[0,1,6]]
        result.columns = ['initial_num', 'comment', 'score']
        # result.sort_values(by='score', ascending=False)
        result = result.values.tolist()
        
        
        
        return result
    else:
        print("Oh this article cannot be processed")
        return

def sub(title, article, comments, likes, dislikes, replys, k):
    article = pr.article_proc(article)
    num = [n+1 for n in range(len(comments))]

    textRank = ut.com_to_trscore(article, comments)
    # values for similarity
    jaccard = []
    cosine = []
    euclidean = []
    for comment in comments:
        jaccard.append(pr.nonset_jac_sim(article,comment))
        tfidf_matrix = pr.tfid_vectorize(article,comment)
        cosine.append(pr.cos_sim(tfidf_matrix))
        euclidean.append(pr.nor_euc_dis(tfidf_matrix))

    val_list1 = cr.norm_data(cr.log_scale(replys, 2))
    val_list2 = cr.norm_data(cr.log_scale(pr.list_calculate(likes,dislikes,k), 2))
    val_list3 = cr.norm_data(pr.list_calculate(textRank, jaccard, 2)) 
    val_list4 = pr.list_calculate(cr.norm_rank(cosine, False), cr.norm_rank(euclidean, True), 2)

    # SHOULD we set this numbers
    a, b, c, d = 2.23, 3.35, 2.92, 4 # default : based on the pre-survey (+ d, max)  
    default = [num, comments, val_list1, val_list2, val_list3, val_list4]
    df = pd.DataFrame(default)
    df = df.T
    df = df.fillna(0)
    df[2] = a*df[2]
    df[3] = b*df[3]
    df[4] = c*df[4]
    df[5] = d*df[5]
    return df

In [13]:
# main() : main(URL, ST1, ST2, ST3, ST4, k)
result, dfIn = main(default_url, 1, 1, 1, 1, -1)

In [29]:
new_df = dfIn[[0,2,3,4,5,6,1]]
new_df.columns = ['initial_num', 'A_reply', 'B_like', 'C_keyword', 'D_similarity', 'score', 'comment']
new_df.sort_values('score', ascending=False).head(20)

final = [dfIn[1].tolist(), dfIn[6].tolist(), dfIn[3].tolist()]
final = pd.DataFrame(final)
final = final.T
final.sort_values(1, ascending=False)
final[[0,2]]

,0,2
0,"무엇 때문 인지 , 이번 대통령님의 임기가 참 길게 느껴집니다.",1
1,민노총집회후 늘었는데 민노총관련확진자 분류하여 발표안하는이정권 이해가안된다 정부는 ...,0.905125
2,3단계로 해라 진짜 스트레스받는다,0.898487
3,외식쿠폰.숙박쿠폰등..공무원들 회식.모임등.. 무능한 정부. 맨날 나라 분열만 조장...,0.854887
4,문재인 진짜 아낌없이 주는 나무네 중국은 혈장검사 검사2회 실시 조치하는데 우리나...,0.802318
...,...,...
96,서울수도권은 대체 왜그러는지 참 이해가안되네..지금은법적으로 마스크도하고 방역지침...,0.25083
97,南韩是共产党。,0.25083
98,코로나 확진자랑 무조건 최대한 멀리 떨어지세요. 코로나바이러스 완치없습니다. 돌연변...,0.25083
99,어디가서 우리나라 시민의식 수준급이라고 하지말자.. 그렇게 모이지말라고 했는데 굳이...,0.25083


In [4]:
# new_df.to_csv("orgin.tsv", sep = '\t')
# new_df.sort_values('score', ascending=False).to_csv('sorted.tsv', sep = '\t')

In [17]:
def default_score(a,b,c,d):
    return (4*a+3*b+2*c+1*d)/(a+b+c+d)
aa = 10,13,24,18 #2.23
bb = 31,26,8,0 #3.35
cc = 24,19,15,7 #2.92
#dd = 
default_score(aa)

TypeError: default_score() missing 3 required positional arguments: 'b', 'c', and 'd'